In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_curve, auc, mean_absolute_error

In [15]:
lc0 = pd.read_csv("LC data原始数据.csv")
lc0.head()

,id,loan_amnt,funded_amnt,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,...,fico_avg,revol_bal,revol_util,total_pymnt,last_pymnt_d,recoveries,loan_length,term_num,return_num,default
0,40715708,14500,14500,36 months,7.89,453.65,A,0,MORTGAGE,58761.0,...,697.0,15449,44.4,14846.090000,2015/5/1,0.0,2.924085,36,0.036707,0
1,113093550,1000,1000,36 months,11.99,33.21,B,1,RENT,65000.0,...,727.0,5154,32.0,1022.724625,2018/2/1,0.0,7.063800,36,0.035226,0
2,8285371,16800,16800,60 months,21.70,461.14,E,10,MORTGAGE,65000.0,...,672.0,17843,87.5,26956.160310,2017/9/1,0.0,45.996838,60,0.151722,0
3,80248236,18000,18000,60 months,23.99,517.72,E,2,MORTGAGE,101000.0,...,692.0,24126,61.4,6416.410000,2017/3/1,1700.0,8.969383,60,-0.118422,1
4,43490782,26000,26000,36 months,12.29,867.18,C,8,RENT,65000.0,...,662.0,40580,71.7,31197.994840,2018/2/1,0.0,35.089016,36,0.066574,0


In [16]:
grade1 = pd.get_dummies(lc0["grade"],drop_first=True)
grade1.head()
lc = pd.concat([lc0,grade1],axis=1)
lc.to_csv("lc0212final.csv")

In [17]:
lc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24977 entries, 0 to 24976
Data columns (total 37 columns):
id                     24977 non-null int64
loan_amnt              24977 non-null int64
funded_amnt            24977 non-null int64
term                   24977 non-null object
int_rate               24977 non-null float64
installment            24977 non-null float64
grade                  24977 non-null object
emp_length             24977 non-null int64
home_ownership         24977 non-null object
annual_inc             24977 non-null float64
verification_status    24977 non-null object
issue_year             24977 non-null int64
loan_status            24977 non-null object
purpose                24977 non-null object
dti                    24977 non-null float64
delinq_2yrs            24977 non-null int64
earliest_cr_line       24977 non-null object
open_acc               24977 non-null int64
pub_rec                24977 non-null int64
fico_range_high        24977 non-null in

In [18]:
lc.columns

Index(['id', 'loan_amnt', 'funded_amnt', 'term', 'int_rate', 'installment',
       'grade', 'emp_length', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_year', 'loan_status', 'purpose', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'open_acc', 'pub_rec',
       'fico_range_high', 'fico_range_low', 'fico_avg', 'revol_bal',
       'revol_util', 'total_pymnt', 'last_pymnt_d', 'recoveries',
       'loan_length', 'term_num', 'return_num', 'default', 'B', 'C', 'D', 'E',
       'F', 'G'],
      dtype='object')

In [19]:
#logistic regression
X = lc[['B', 'C', 'D', 'E', 'F', 'G', 'funded_amnt', 'annual_inc', 'issue_year',
       'dti', 'fico_avg', 'emp_length', 'revol_bal', 'term_num']]
y = lc['default']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=3)

X_train1=sm.tools.tools.add_constant(X_train)
logit=sm.Logit(y_train,X_train1)

result=logit.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.457578
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                default   No. Observations:                17483
Model:                          Logit   Df Residuals:                    17468
Method:                           MLE   Df Model:                           14
Date:                Thu, 13 Feb 2020   Pseudo R-squ.:                 0.08081
Time:                        15:19:40   Log-Likelihood:                -7999.8
converged:                       True   LL-Null:                       -8703.2
                                        LLR p-value:                6.066e-292
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const        -147.3046     28.146     -5.234      0.000    -202.470     -92.139
B               0.5327      0.090      5.931      0.000       0.357       0.709
C               0.9332      0.091     10.306      0.000       0.756       1.111
D               1.2677      0.097     13.118      0.000       1.078       1.457
E               1.4968      0.108     13.886      0.000       1.286       1.708
F               1.6163      0.136     11.911      0.000       1.350       1.882
G               1.6697      0.204      8.192      0.000       1.270       2.069
funded_amnt  1.421e-05   2.95e-06      4.821      0.000    8.43e-06       2e-05
annual_inc  -2.958e-06   6.04e-07     -4.899      0.000   -4.14e-06   -1.77e-06
issue_year      0.0736      0.014      5.268      0.000       0.046       0.101
dti             0.0199      0.003      7.881      0.000       0.015       0.025
fico_avg       -0.0059      0.001     -6.935      0.000      -0.008      -0.004
emp_length     -0.0196      0.005     -3.720      0.000      -0.030      -0.009
revol_bal   -2.805e-06   1.27e-06     -2.201      0.028    -5.3e-06   -3.07e-07
term_num        0.0164      0.002      7.717      0.000       0.012       0.021
===============================================================================
"""

In [35]:
B1=lc["B"]*0.5327
C1=lc["C"]*0.9332
D1=lc["D"]*1.2677
E1=lc["E"]*1.4968
F1=lc["F"]*1.6163
G1=lc["G"]*1.6697
funded_amnt2=lc["funded_amnt"]*(1.421e-05)
annual_inc2=lc["annual_inc"]*(-2.958e-06)
issue_year2=lc["issue_year"]*0.0736
dti2=lc["dti"]*0.0199
fico_avg2=lc["fico_avg"]*(-0.0059)
emp_length2=lc["emp_length"]*(-0.0196)
revol_bal2=lc["revol_bal"]*(-2.805e-06)
term_num2=lc["term_num"]*0.0164
int_rate2=lc["int_rate"]
default2=lc["default"]
id2=lc["id"]
loan_amnt2=lc["loan_amnt"]
grade=lc["grade"]
total_pymnt=lc["total_pymnt"]
last_pymnt_d=lc["last_pymnt_d"]
recoveries=lc["recoveries"]
return_num=lc["return_num"]
loan_length=lc["loan_length"]
llc2=pd.concat([id2,grade,total_pymnt,recoveries,loan_length,loan_amnt2,last_pymnt_d,return_num,B1,C1,D1,E1,F1,G1,dti2,fico_avg2,emp_length2,revol_bal2,term_num2,funded_amnt2,annual_inc2,issue_year2,int_rate2,default2],axis=1)
llc2.to_csv("0212llc2.csv")
lc2=pd.read_csv("0212llc2.csv")# 'funded_amnt.1'是乘上系数了的
lc2.columns

Index(['Unnamed: 0', 'id', 'grade', 'total_pymnt', 'recoveries', 'loan_length',
       'loan_amnt', 'last_pymnt_d', 'return_num', 'B', 'C', 'D', 'E', 'F', 'G',
       'dti', 'fico_avg', 'emp_length', 'revol_bal', 'term_num', 'funded_amnt',
       'annual_inc', 'issue_year', 'int_rate', 'default'],
      dtype='object')

In [36]:
X = lc2[['loan_amnt', 'grade',"loan_length",'total_pymnt', 'last_pymnt_d',
       'recoveries', 'return_num', 'B', 'C', 'D', 'E', 'F', 'G', 'dti',
       'fico_avg', 'emp_length', 'revol_bal', 'term_num', 'funded_amnt',
       'annual_inc', 'issue_year', 'int_rate']]
y = lc2['default']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=3)

w = -147.3046+X_test['B']+X_test['C']+X_test['D']+X_test['E']+X_test['F']+X_test['G']+X_test['funded_amnt']+X_test['annual_inc']+X_test['issue_year']+X_test['dti']+X_test['fico_avg']+X_test['emp_length']+X_test['revol_bal']+X_test['term_num']
y_scores = np.exp(w)/(np.exp(w)+1)
data1 = pd.Series(y_scores)
dataframe = pd.DataFrame({'data1_name':data1})
dataframe.to_csv('probability_0213final.csv',mode = 'a',index=0,header=1)
X_test.to_csv("X_test4.csv")
y_test.to_csv("y_test4.csv")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [37]:
y_scores.describe()

count    7.494000e+03
mean     2.081719e-01
std      1.177936e-01
min      1.453147e-11
25%      1.189750e-01
50%      1.867506e-01
75%      2.788073e-01
max      9.756844e-01
dtype: float64

In [38]:
w.describe()

count    7494.000000
mean       -1.506993
std         0.867978
min       -24.954705
25%        -2.002172
50%        -1.471264
75%        -0.950386
max         3.692019
dtype: float64

In [27]:
lc3 = pd.read_csv("X_test2.csv")
lc3.columns

Index(['Unnamed: 0', 'loan_amnt', 'grade', 'total_pymnt', 'last_pymnt_d',
       'recoveries', 'return_num', 'B', 'C', 'D', 'E', 'F', 'G', 'dti',
       'fico_avg', 'emp_length', 'revol_bal', 'term_num', 'funded_amnt',
       'annual_inc', 'issue_year', 'int_rate', 'default', 'def_proba', 'std',
       'return_num2'],
      dtype='object')

In [12]:
lc4 = pd.read_csv("X_test2.csv")
lc4.head()

,Unnamed: 0,loan_amnt,grade,total_pymnt,last_pymnt_d,recoveries,return_num,B,C,D,...,emp_length,revol_bal,term_num,funded_amnt,annual_inc,issue_year,int_rate,default,def_proba,std
0,17360,10500,C,12600.233910,2018/3/1,0.0,0.066295,0.0000,0.9332,0.0000,...,-0.1764,-0.048151,0.5904,0.149205,-0.207060,148.3040,12.29,0,0.175837,NaN
1,10661,2400,B,2426.939779,2018/1/1,0.0,0.034340,0.5327,0.0000,0.0000,...,-0.1372,-0.008917,0.5904,0.034104,-0.059160,148.4512,9.93,0,0.103124,NaN
2,9317,10000,B,10666.220000,2016/9/1,0.0,0.044112,0.5327,0.0000,0.0000,...,-0.0392,-0.039769,0.5904,0.142100,-0.221850,148.3040,8.18,0,0.130045,NaN
3,22469,28825,D,22061.310000,2017/9/1,0.0,-0.027205,0.0000,0.0000,1.2677,...,-0.1960,-0.092414,0.5904,0.409603,-0.212976,148.3040,15.41,1,0.329238,NaN
4,18656,13025,F,8705.500000,2016/2/1,0.0,-0.049468,0.0000,0.0000,0.0000,...,-0.1960,-0.054950,0.9840,0.185085,-0.266220,148.2304,24.50,1,0.460788,NaN


In [14]:
lc4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7494 entries, 0 to 7493
Data columns (total 25 columns):
Unnamed: 0      7494 non-null int64
loan_amnt       7494 non-null int64
grade           7494 non-null object
total_pymnt     7494 non-null float64
last_pymnt_d    7494 non-null object
recoveries      7494 non-null float64
return_num      7494 non-null float64
B               7494 non-null float64
C               7494 non-null float64
D               7494 non-null float64
E               7494 non-null float64
F               7494 non-null float64
G               7494 non-null float64
dti             7494 non-null float64
fico_avg        7494 non-null float64
emp_length      7494 non-null float64
revol_bal       7494 non-null float64
term_num        7494 non-null float64
funded_amnt     7494 non-null float64
annual_inc      7494 non-null float64
issue_year      7494 non-null float64
int_rate        7494 non-null float64
default         7494 non-null int64
def_proba       7494 non-nu

In [13]:
group = lc4.groupby("grade").describe()
group["return_num"]

,count,mean,std,min,25%,50%,75%,max
grade,,,,,,,,
A,1218.0,0.037748,0.031307,-0.192947,0.040363,0.044657,0.048767,0.067719
B,2174.0,0.036807,0.052777,-0.192613,0.042534,0.053747,0.061757,0.102444
C,2110.0,0.032182,0.068374,-0.192400,0.038696,0.059000,0.071282,0.124387
D,1120.0,0.029313,0.078910,-0.191980,-0.006785,0.061674,0.082591,0.142536
E,609.0,0.020345,0.092184,-0.193580,-0.068668,0.059259,0.090668,0.159075
F,215.0,0.023307,0.099833,-0.185626,-0.066404,0.055828,0.102488,0.174167
G,48.0,-0.011598,0.108995,-0.177996,-0.128086,0.013683,0.084311,0.157449
